# INTRINSIC VALUE CALCULATOR

1. PORTFOLIO HEALTH CHECK
2. STOCK PE HISTOGRAM
3. STOCK PR DISTRIBUTION
4. PE VS RETURN

In [2]:
import akshare as ak

import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datetime import datetime

## PORTFOLIO HEALTH CHECK
1. portfolio earning = eps ttm x shares
2. portfolio net asset = bps x shares
3. earning yield = portfolio earning / cost
4. net asset yield = portfolio net asset / cost
5. roe = portfolio earning / portfolio net asset

In [3]:
# UPDATE THE DICTIONARY BELOW
investment_dict = {
    '20251212': {
        "stock_code": ["600519", "000858", "600938", "000333", "601088", "300866", "600900"],
        "cost_per_share": [1482.8976, 139.3360, 27.4303, 76.5841, 42.1904, 94.5264, 27.1260], 
        "shares": [100, 700, 2600, 700, 1000, 400, 1200]
    }, 
    '20251220': {
        "stock_code": ["600519", "000858", "600938", "000333", "600926", "300866", "600900"],
        "cost_per_share": [1458.9406, 136.7580, 27.4303, 76.5841, 15.1237, 94.5264, 27.1260], 
        "shares": [100, 700, 2600, 700, 2700, 400, 1200]
    }
    
}

# CHANGE THE DATE HERE
portfolio_date = '20251220'

STOCK_CODES = investment_dict[portfolio_date]['stock_code']
STOCK_SYMBOLS_1 = [code + ".SH" if code.startswith("6") else code + ".SZ" for code in STOCK_CODES ]
STOCK_SYMBOLS_2 = ['sh' + code if code.startswith("6") else 'sz' + code for code in STOCK_CODES]
STOCK_SYMBOLS_3 = ['SH' + code if code.startswith("6") else 'SZ' + code for code in STOCK_CODES]

investment_df = pd.DataFrame(investment_dict[portfolio_date])
investment_df

,stock_code,cost_per_share,shares
0,600519,1458.9406,100
1,000858,136.7580,700
2,600938,27.4303,2600
3,000333,76.5841,700
4,600926,15.1237,2700
5,300866,94.5264,400
6,600900,27.1260,1200


In [28]:
today = datetime.today().strftime('%Y%m%d')

# for season_gap in tqdm(range(12)): # season gap = 0 is the most recent season
dfs = []
for stock_code, symbol in tqdm(zip(STOCK_CODES, STOCK_SYMBOLS_1)):
    # load the eps and roe data
    eps_roe_df = ak.stock_financial_analysis_indicator_em(symbol=symbol, indicator="按报告期")
    eps_roe_df = eps_roe_df[["REPORT_DATE", "REPORT_TYPE", "REPORT_DATE_NAME", 
                            "EPSJB", "BPS", "ROEJQ"]]

    # rename the columns
    eps_roe_df.columns = ["date", "report_type", "report_date_type", "eps", "bps", "roe"]

    # calculate eps ttm
    eps_roe_df['eps_season'] = eps_roe_df['eps'].diff(-1)
    eps_roe_df['eps_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                        eps_roe_df['eps'], eps_roe_df['eps_season'])
    eps_roe_df['eps_ttm'] = eps_roe_df['eps_season'].rolling(4).sum().shift(-3)

    # calculate roe ttm
    eps_roe_df['roe_season'] = eps_roe_df['roe'].diff(-1)
    eps_roe_df['roe_season'] = np.where(eps_roe_df['report_type'] == '一季报', 
                                        eps_roe_df['roe'], eps_roe_df['roe_season'])
    eps_roe_df['roe_ttm'] = eps_roe_df['roe_season'].rolling(4).sum().shift(-3)

    # eps_roe_df = eps_roe_df.iloc[season_gap]
    eps_roe_df = eps_roe_df[['date', 'report_type', 'report_date_type', 'eps_ttm', 'bps', 'roe_ttm']]
    eps_roe_df['stock_code'] = symbol[:6]
    eps_roe_df.to_csv(f"../data/input/portfolio/eps_roe_{stock_code}_{portfolio_date}_{today}.csv", index=False)
    dfs.append(eps_roe_df)

7it [00:03,  1.99it/s]


In [29]:
portfolio_dfs = []

for season_gap in tqdm(range(12)): # season gap = 0 is the most recent season
    stock_dfs = []
    for df in dfs:
        stock_df = df.iloc[season_gap]
        stock_dfs.append(stock_df)
    stock_dfs = pd.DataFrame(stock_dfs)
    portfolio_df = pd.merge(investment_df, stock_dfs, on='stock_code', how='left', validate="1:1")

    # calculate portfolio metrics
    portfolio_df = portfolio_df.eval("cost = cost_per_share * shares")
    portfolio_df = portfolio_df.eval("porfolio_earning = eps_ttm * shares")
    portfolio_df = portfolio_df.eval("porfolio_net_asset = bps * shares")

    portfolio_df.loc["portfolio", "cost"] = portfolio_df["cost"].sum()
    portfolio_df.loc["portfolio", "porfolio_earning"] = portfolio_df["porfolio_earning"].sum()
    portfolio_df.loc["portfolio", "porfolio_net_asset"] = portfolio_df["porfolio_net_asset"].sum()
    portfolio_df.loc["portfolio", "roe_ttm"] = portfolio_df.loc["portfolio", "porfolio_earning"] / portfolio_df.loc["portfolio", "porfolio_net_asset"] * 100

    portfolio_df = portfolio_df.eval("earning_yield = porfolio_earning / cost")
    portfolio_df = portfolio_df.eval("net_asset_yield = porfolio_net_asset / cost")

    portfolio_df.loc["portfolio", "stock_code"] = "portfolio" + "_season_gap_" + str(season_gap)
    portfolio_df.loc["portfolio", "date"] = portfolio_df.iloc[0, 3] # column 3 is date
    portfolio_df.loc["portfolio", "report_type"] = portfolio_df.iloc[0, 4] # column 4 is report_type

    portfolio_dfs.append(portfolio_df)

portfolio_dfs = pd.concat(portfolio_dfs)
portfolio_dfs.to_csv(f"../data/processed/portfolio/portfolio_intrinsic_value_over_seasons_{portfolio_date}_{today}.csv", index=False)
portfolio_dfs.loc["portfolio"]

100%|██████████| 12/12 [00:00<00:00, 165.97it/s]


,stock_code,cost_per_share,shares,date,report_type,report_date_type,eps_ttm,bps,roe_ttm,cost,porfolio_earning,porfolio_net_asset,earning_yield,net_asset_yield
portfolio,portfolio_season_gap_0,NaN,NaN,2025-09-30 00:00:00,三季报,NaN,NaN,NaN,19.271059,477748.06,34116.08,177032.723241,0.071410,0.370557
portfolio,portfolio_season_gap_1,NaN,NaN,2025-06-30 00:00:00,中报,NaN,NaN,NaN,20.386954,477748.06,35222.01,172767.399467,0.073725,0.361629
portfolio,portfolio_season_gap_2,NaN,NaN,2025-03-31 00:00:00,一季报,NaN,NaN,NaN,19.891811,477748.06,35484.98,178389.895072,0.074276,0.373397
portfolio,portfolio_season_gap_3,NaN,NaN,2024-12-31 00:00:00,年报,NaN,NaN,NaN,20.436307,477748.06,34545.62,169040.420495,0.072309,0.353828
portfolio,portfolio_season_gap_4,NaN,NaN,2024-09-30 00:00:00,三季报,NaN,NaN,NaN,20.434820,477748.06,34053.32,166643.602617,0.071279,0.348811
portfolio,portfolio_season_gap_5,NaN,NaN,2024-06-30 00:00:00,中报,NaN,NaN,NaN,20.823666,477748.06,33083.21,158873.130040,0.069248,0.332546
portfolio,portfolio_season_gap_6,NaN,NaN,2024-03-31 00:00:00,一季报,NaN,NaN,NaN,19.310022,477748.06,31793.32,164646.728410,0.066548,0.344631
portfolio,portfolio_season_gap_7,NaN,NaN,2023-12-31 00:00:00,年报,NaN,NaN,NaN,19.758931,477748.06,30403.78,153873.610517,0.063640,0.322081
portfolio,portfolio_season_gap_8,NaN,NaN,2023-09-30 00:00:00,三季报,NaN,NaN,NaN,20.303788,477748.06,30217.34,148826.122363,0.063250,0.311516
portfolio,portfolio_season_gap_9,NaN,NaN,2023-06-30 00:00:00,中报,NaN,NaN,NaN,20.563415,477748.06,29347.54,142717.245330,0.061429,0.298729


In [30]:
portfolio_cost = portfolio_dfs.loc[portfolio_dfs.stock_code == "portfolio_season_gap_0", "cost"].values[0]
portfolio_earning = portfolio_dfs.loc[portfolio_dfs.stock_code == "portfolio_season_gap_0", "porfolio_earning"].values[0]
portfolio_net_asset = portfolio_dfs.loc[portfolio_dfs.stock_code == "portfolio_season_gap_0", "porfolio_net_asset"].values[0]
portfolio_earning_yield = portfolio_dfs.loc[portfolio_dfs.stock_code == "portfolio_season_gap_0", "earning_yield"].values[0]
portfolio_net_asset_yield = portfolio_dfs.loc[portfolio_dfs.stock_code == "portfolio_season_gap_0", "net_asset_yield"].values[0]
print(portfolio_cost, portfolio_earning, portfolio_net_asset, portfolio_earning_yield, portfolio_net_asset_yield)

477748.06 34116.08 177032.7232408684 0.07141019055106158 0.37055665540717925


## EMAIL SENDER

In [31]:
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.header import Header
from smtplib import SMTP_SSL
from datetime import datetime

import pandas as pd

import os 
from dotenv import load_dotenv

In [32]:
def send_mail(receiver='', mail_title='', mail_content=''):
    # ssl login
    smtp = SMTP_SSL(host_server)
    # set_debuglevel() for debug, 1 enable debug, 0 for disable
    # smtp.set_debuglevel(1)
    smtp.ehlo(host_server)
    smtp.login(sender_mail, sender_passcode)

    # construct message
    msg = MIMEText(mail_content, "plain", 'utf-8')
    msg["Subject"] = Header(mail_title, 'utf-8')
    msg["From"] = sender_mail
    msg["To"] = receiver
    smtp.sendmail(sender_mail, receiver, msg.as_string())
    smtp.quit()

In [ ]:
load_dotenv()
today = datetime.now().strftime('%Y%m%d')

# qq mail sending server
host_server = os.getenv("SMTP_SERVER")
sender_mail = os.getenv("SENDER_EMAIL")
sender_passcode = os.getenv("EMAIL_AUTH_CODE")

# receiver mail
receiver = os.getenv("RECEIVER_EMAIL")
# mail title
mail_title = f'Portfolio Intrinsic Value by {portfolio_date}'
# mail contents
mail_content = f'The portfolio intrinsic value by {portfolio_date} are: \n' + \
f'portfolio_cost: {portfolio_cost} \n' + \
f'portfolio_earning: {portfolio_earning} \n' + \
f'portfolio_net_asset: {portfolio_net_asset} \n' + \
f'portfolio_earning_yield: {portfolio_earning_yield} \n' + \
f'portfolio_net_asset_yield: {portfolio_net_asset_yield} \n'

send_mail(receiver=receiver,mail_title=mail_title,mail_content=mail_content)
print('Email sent successfully.')

Email sent successfully.
